- Усложнить и запустить для выявления нескольких компаний
- Усложнить и дать задачу угадывать компанию в том числе

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score

## Подготовка промпта

In [78]:
issuers_df = pd.read_excel('../data/issuers.xlsx', index_col = 0)
mentions_df = pd.read_csv('../data/mentions.csv', index_col = 0)
mentions_texts_df = pd.read_pickle('../data/mentions texts.pickle')
sentiment_df = pd.read_csv('../data/sentiment.csv', index_col = 0)
sentiment_texts_df = pd.read_pickle('../data/sentiment_texts.pickle')

In [79]:
# объединим дубли
sentiment_texts_df = sentiment_texts_df.groupby(['ChannelID', 'MessageID', 'issuerid', 'MessageText'], as_index = False)['SentimentScore'].mean()

# добавим названия компаний
sentiment_texts_df = sentiment_texts_df.merge(issuers_df[['issuerid', 'EMITENT_FULL_NAME']], on = ['issuerid'], how = 'left', validate = 'm:1')

def getScoreText(x):
    if x <= 1:
        return 'Сильно негативная'
    elif x <= 2:
        return 'Негативная'
    elif x <= 3:
        return 'Нейтральная'
    elif x <= 4:
        return 'Позитивная'
    else:
        return 'Сильно позитивная'

# добавим текстовое обозначение сентименту
sentiment_texts_df['SentimentScoreText'] = sentiment_texts_df['SentimentScore'].apply(getScoreText)

In [112]:
def getSample(df, add_taget = True):
    row = df.sample(1)
    
    text = ''
    text += 'Новость: ' + row['MessageText'].values[0]
    text += '\n'
    text += 'Компания: ' + row['EMITENT_FULL_NAME'].values[0]
    if add_taget:
        text += '\n'
        text += 'Оценка: ' + row['SentimentScoreText'].values[0] + '</s>'
    else:
        text += '\n'
        text += 'Оценка: '
        
    
    return text, row

In [104]:
def getPrompt(df, num_examples = 1):
    
    prompt = 'Ты работаешь финансовым аналитиком. Твоя задача - оценить сентимент по каждой из компаний, которые есть в тексте новости.\n'\
        + 'Сентимент может принимать значения: "Сильно негативная", "Негативная", "Нейтральная", "Позитивная", "Сильно позитивная"\n'\
        + 'Вот примеры твоих работ.\n\n'
    
    for _ in range(num_examples):
        text, _ = getSample(df)
        prompt += text+ '\n\n'
    
    # get target
    target_text, target_row = getSample(df, add_taget = False)
    prompt += target_text
    
    return prompt, target_row['SentimentScoreText']

In [116]:
test_prompt, test_target = getPrompt(sentiment_texts_df, num_examples=3)
print(test_prompt)

Ты работаешь финансовым аналитиком. Твоя задача - оценить сентимент по каждой из компаний, которые есть в тексте новости.
Сентимент может принимать значения: "Сильно негативная", "Негативная", "Нейтральная", "Позитивная", "Сильно позитивная"
Вот примеры твоих работ.

Новость: ⚡️⚡️⚡️ Недавний рост Urals и цен на нефтепродукты делает Татнефть (TATN) не менее инвестиционно-привлекательной компанией, чем Лукойл (LKOH) и Роснефть (ROSN), сообщает источник РДВ. #аналитика  👉 Потенциал роста: 26% до справедливой цены в 733 руб./акция  @AK47pfl
Компания: Публичное акционерное общество "Татнефть" имени В.Д. Шашина
Оценка: Сильно позитивная</s>

Новость: 🇷🇺 $VKCO #банки  ВТБ до конца апреля запустит онлайн-банк в VK Мессенджере, который позволит совершать большинство популярных операций внутри социальной сети  — Печатников - Интерфакс
Компания: Mail.ru Group Limited (Мэйл.ру Груп Лимитед) (эмитент депозитарных расписок - Citibank N.A. (Ситибэнк Эн.Эй))
Оценка: Позитивная</s>

Новость: "Среди дол

## Загрузка модели и предикт

In [105]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration

/opt/conda/envs/gagarin/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [107]:
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [118]:
input_ids=torch.tensor([tokenizer.encode(test_prompt)]).to(device)
outputs=model.generate(input_ids, do_sample = True, temperature=0.1, num_beams =2, early_stopping=True,
                      repetition_penalty = 2.5, max_new_tokens=64)
predict = tokenizer.decode(outputs[0][1:])
predict

' 📌 Не является инвестиционной рекомендацией ИнвестТема\nКомпания: Fix Price (Fix Price Holdings Limited)\nОценка: Позитивная, нейтральная, негативная.\n\nНовость: "ВТБ" и "Газпромбанк" запустили онлайн-банк'